In [1]:
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from scipy.stats import multivariate_normal
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Instantiate a QuantBook.
qb = QuantBook()

# Select the desired tickers for research.
asset = "SPX"

train_cutoff_date = datetime(2024,12, 31)

# Call the AddIndex method with the tickers, and its corresponding resolution. Then store their Symbols. Resolution.Minute is used by default. 
qb.add_index(asset, Resolution.MINUTE)

# Call the history method with qb.securities.keys for all tickers, time argument(s), and resolution to request historical data for the symbol.
history = qb.history(qb.securities.keys(), datetime(2019, 1, 1), datetime(2025, 11, 10), Resolution.DAILY)

# Get the close price daily return.
close = history['close'].unstack(level=0)

# Call pct_change to obtain the daily return
returns = close.pct_change().iloc[1:]

# Initialize the HMM, then fit by the daily return data. Note that we're using varinace as switching regime, so switching_variance is set as True.
model = MarkovRegression(returns, k_regimes=2, switching_variance=True).fit()
print(model.summary())

In [2]:
# Get the regime as a column, 1 as Low Variance Regime, 2 as High Variance Regime.
regime = pd.Series(model.smoothed_marginal_probabilities.values.argmax(axis=1)+1, 
                      index=returns.index, name='regime')
df_1 = close.loc[returns.index][regime == 1]
df_2 = close.loc[returns.index][regime == 2]

# Get the mean and covariance matrix of the 2 regimes, assume 0 covariance between the two.
mean = np.array([returns.loc[df_1.index].mean(), returns.loc[df_2.index].mean()])
cov = np.array([[returns.loc[df_1.index].var()[0], 0], [0, returns.loc[df_2.index].var()[0]]])

# Fit a 2-dimensional multivariate normal distribution by the 2 means and covriance matrix.
dist = multivariate_normal(mean=mean.flatten(), cov=cov)
mean_1, mean_2 = mean[0], mean[1]
sigma_1, sigma_2 = cov[0,0], cov[1,1]


# Plot the probability of data in different regimes.
fig, axes = plt.subplots(2, figsize=(15, 10))
ax = axes[0]
ax.plot(model.smoothed_marginal_probabilities[0])
ax.set(title='Smoothed probability of Low Variance Regime')
ax = axes[1]
ax.plot(model.smoothed_marginal_probabilities[1])
ax.set(title='Smoothed probability of High Variance Regime')
fig.tight_layout()
plt.show()

# Plot the series into regime-wise.
df_1.index = pd.to_datetime(df_1.index)
df_1 = df_1.sort_index()
df_2.index = pd.to_datetime(df_2.index)
df_2 = df_2.sort_index()

plt.figure(figsize=(15, 10))
plt.scatter(df_1.index, df_1, color='blue', label="Low Variance Regime")
plt.scatter(df_2.index, df_2, color='red', label="High Variance Regime")

# Add shaded region with label
plt.axvspan('2024-07-01', '2025-07-01', alpha=0.2, color='green', zorder=-1, label="Backtesting Period")

plt.title("SPY Price")
plt.ylabel("Price ($)")
plt.xlabel("Date")
plt.legend()
plt.show()


In [3]:
# 1. Distribution of returns for each regime
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

ax = axes[0]
returns_regime_1 = returns.loc[df_1.index]
ax.hist(returns_regime_1.values.flatten(), bins=50, alpha=0.7, color='blue', edgecolor='black')
ax.axvline(mean_1[0], color='darkblue', linestyle='--', linewidth=2, label=f'Mean: {mean_1[0]:.4f}')
ax.set_title('Distribution of Returns - Low Variance Regime')
ax.set_xlabel('Returns')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid(alpha=0.3)

ax = axes[1]
returns_regime_2 = returns.loc[df_2.index]
ax.hist(returns_regime_2.values.flatten(), bins=50, alpha=0.7, color='red', edgecolor='black')
ax.axvline(mean_2[0], color='darkred', linestyle='--', linewidth=2, label=f'Mean: {mean_2[0]:.4f}')
ax.set_title('Distribution of Returns - High Variance Regime')
ax.set_xlabel('Returns')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid(alpha=0.3)

fig.tight_layout()
plt.show()

plt.figure(figsize=(15, 6))
plt.hist(returns_regime_1.values.flatten(), bins=50, alpha=0.5, color='blue', 
         label=f'Low Variance (σ²={sigma_1:.6f})', edgecolor='black')
plt.hist(returns_regime_2.values.flatten(), bins=50, alpha=0.5, color='red', 
         label=f'High Variance (σ²={sigma_2:.6f})', edgecolor='black')
plt.axvline(mean_1[0], color='darkblue', linestyle='--', linewidth=2)
plt.axvline(mean_2[0], color='darkred', linestyle='--', linewidth=2)
plt.title('Comparison of Return Distributions by Regime')
plt.xlabel('Returns')
plt.ylabel('Frequency')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [4]:
# 2. Analyze regime switching duration and frequency
regime_changes = regime.diff().fillna(0)
switches = regime_changes[regime_changes != 0]

# Calculate duration of each regime period
regime_periods = []
current_regime = regime.iloc[0]
start_date = regime.index[0]
duration = 1

for i in range(1, len(regime)):
    if regime.iloc[i] == current_regime:
        duration += 1
    else:
        # Regime changed, store the previous period
        regime_periods.append({
            'regime': current_regime,
            'start_date': start_date,
            'end_date': regime.index[i-1],
            'duration_days': duration
        })
        # Start new period
        current_regime = regime.iloc[i]
        start_date = regime.index[i]
        duration = 1

# Add the last period
regime_periods.append({
    'regime': current_regime,
    'start_date': start_date,
    'end_date': regime.index[-1],
    'duration_days': duration
})

regime_durations_df = pd.DataFrame(regime_periods)

# Summary statistics
print(f"\n{'='*60}")
print("REGIME SWITCHING ANALYSIS")
print(f"{'='*60}")
print(f"\nTotal number of regime switches: {len(switches)}")
print(f"Total periods analyzed: {len(regime_durations_df)}")
print(f"\nRegime 1 (Low Variance):")
regime_1_durations = regime_durations_df[regime_durations_df['regime'] == 1]['duration_days']
print(f"  Number of periods: {len(regime_1_durations)}")
print(f"  Average duration: {regime_1_durations.mean():.2f} days")
print(f"  Median duration: {regime_1_durations.median():.2f} days")
print(f"  Min duration: {regime_1_durations.min()} days")
print(f"  Max duration: {regime_1_durations.max()} days")

print(f"\nRegime 2 (High Variance):")
regime_2_durations = regime_durations_df[regime_durations_df['regime'] == 2]['duration_days']
print(f"  Number of periods: {len(regime_2_durations)}")
print(f"  Average duration: {regime_2_durations.mean():.2f} days")
print(f"  Median duration: {regime_2_durations.median():.2f} days")
print(f"  Min duration: {regime_2_durations.min()} days")
print(f"  Max duration: {regime_2_durations.max()} days")

fig, ax = plt.subplots(figsize=(8, 6))

ax.boxplot([regime_1_durations, regime_2_durations], 
           labels=['Low Variance', 'High Variance'],
           patch_artist=True,
           boxprops=dict(facecolor='lightblue', alpha=0.7),
           medianprops=dict(color='red', linewidth=2))

ax.set_title('Regime Duration Comparison')
ax.set_ylabel('Duration (days)')
ax.grid(alpha=0.3)

fig.tight_layout()
plt.show()

In [5]:
# Calculate confidence metrics for regime predictions
regime_confidence = model.smoothed_marginal_probabilities.max(axis=1)
regime_uncertainty = 1 - regime_confidence  # or you could use entropy

# Add confidence to our regime series
regime_with_confidence = pd.DataFrame({
    'regime': regime,
    'confidence': regime_confidence,
    'prob_regime_1': model.smoothed_marginal_probabilities[0],
    'prob_regime_2': model.smoothed_marginal_probabilities[1]
})

print(f"\nOverall confidence statistics:")
print(f"  Mean confidence: {regime_confidence.mean():.4f}")
print(f"  Median confidence: {regime_confidence.median():.4f}")
print(f"  Min confidence: {regime_confidence.min():.4f}")
print(f"  Max confidence: {regime_confidence.max():.4f}")

# Identify low confidence periods (e.g., confidence < 0.7)
low_confidence_threshold = 0.7
low_confidence_periods = regime_with_confidence[regime_with_confidence['confidence'] < low_confidence_threshold]
print(f"\nPeriods with confidence < {low_confidence_threshold}: {len(low_confidence_periods)} days ({len(low_confidence_periods)/len(regime)*100:.2f}%)")

# Analyze confidence around regime switches
regime_changes_idx = regime_changes[regime_changes != 0].index
switch_window = 5  # days before and after switch
print(f"\n Confidence at switch statistics:")
switch_confidence_analysis = []
for switch_date in regime_changes_idx:
    switch_loc = regime.index.get_loc(switch_date)
    start_idx = max(0, switch_loc - switch_window)
    end_idx = min(len(regime), switch_loc + switch_window + 1)
    
    window_dates = regime.index[start_idx:end_idx]
    window_confidence = regime_confidence.iloc[start_idx:end_idx]
    
    switch_confidence_analysis.append({
        'switch_date': switch_date,
        'from_regime': regime.iloc[switch_loc - 1] if switch_loc > 0 else None,
        'to_regime': regime.iloc[switch_loc],
        'confidence_at_switch': regime_confidence.iloc[switch_loc],
        'avg_confidence_before': window_confidence.iloc[:switch_window].mean() if switch_loc >= switch_window else None,
        'avg_confidence_after': window_confidence.iloc[switch_window:].mean() if switch_loc < len(regime) - switch_window else None
    })

switch_confidence_df = pd.DataFrame(switch_confidence_analysis)

print(f"\nAverage confidence at switch points: {switch_confidence_df['confidence_at_switch'].mean():.4f}")
print(f"\nMedian confidence at switch points: {switch_confidence_df['confidence_at_switch'].median():.4f}")
print(f"Max confidence at switch points: {switch_confidence_df['confidence_at_switch'].max():.4f}")
print(f"Minimum confidence at switch points: {switch_confidence_df['confidence_at_switch'].min():.4f}")
fig, ax = plt.subplots(figsize=(8, 6))

ax.hist(switch_confidence_df['confidence_at_switch'], bins=20, 
        color='purple', alpha=0.7, edgecolor='black')
ax.axvline(switch_confidence_df['confidence_at_switch'].mean(), 
           color='red', linestyle='--', linewidth=2, 
           label=f"Mean: {switch_confidence_df['confidence_at_switch'].mean():.3f}")
ax.axvline(switch_confidence_df['confidence_at_switch'].median(), 
           color='orange', linestyle='--', linewidth=2, 
           label=f"Median: {switch_confidence_df['confidence_at_switch'].median():.3f}")
ax.set_title('Distribution of Confidence at Regime Switches')
ax.set_xlabel('Confidence')
ax.set_ylabel('Number of Switches')
ax.legend()
ax.grid(alpha=0.3)

fig.tight_layout()
plt.show()


In [6]:
from sklearn.covariance import EmpiricalCovariance
def calculate_turbulence(returns_df, lookback=252*3):
    """Calculate Kritzman turbulence using Mahalanobis distance"""
    
    # Use sector returns only (not SPY)
    sector_returns = returns_df[[col for col in returns_df.columns if col != 'SPY']]
    
    turbulence = []
    
    for i in range(lookback, len(sector_returns)):
        # Historical window
        hist_data = sector_returns.iloc[i-lookback:i].values
        current_returns = sector_returns.iloc[i].values
        
        # Mean and covariance
        mu = hist_data.mean(axis=0)
        cov_estimator = EmpiricalCovariance()
        cov_estimator.fit(hist_data)
        cov = cov_estimator.covariance_
        
        # Mahalanobis distance
        diff = current_returns - mu
        try:
            turb_score = diff @ np.linalg.inv(cov) @ diff
        except:
            turb_score = diff @ np.linalg.pinv(cov) @ diff
        
        turbulence.append(turb_score)
    
    # Create series with NaN padding
    turbulence_series = pd.Series(
        [np.nan] * lookback + turbulence,
        index=sector_returns.index
    )
    
    return turbulence_series

In [7]:
sectors = ['XLK', 'XLF', 'XLE', 'XLV', 'XLY', 'XLP', 'XLI', 'XLU', 'XLB']

for sector in sectors:
    qb.add_equity(sector)

sectors_history = qb.history(sectors, datetime(2019, 1, 1), datetime(2025, 11, 10), Resolution.DAILY)

# Get the close price daily return.
sectors_close = sectors_history['close'].unstack(level=0)

# Call pct_change to obtain the daily return
sectors_returns = sectors_close.pct_change().iloc[1:]

In [8]:
turbulence = calculate_turbulence(sectors_returns.dropna(), lookback=252*1)

In [9]:
crisis_events = {
    'COVID-19': ('2020-02-20', '2020-04-01'),
    'Fed Tightening': ('2022-01-01', '2022-10-31'),
    'Tariff': ('2025-04-02', '2025-05-16')
}

def evaluate_crisis_detection(states, crisis_events):
    """Evaluate how well model detects known crises"""
    results = {}
    for event_name, (start, end) in crisis_events.items():
        mask = (states.index >= start) & (states.index <= end)
        if mask.sum() > 0:
            detection_rate = states[mask].mean() * 100
            results[event_name] = f"{detection_rate:.1f}%"
        else:
            results[event_name] = "N/A"
    return results

def calculate_regime_metrics(states):
    """Calculate stability metrics"""
    switches = (states.diff() != 0).sum()
    years = len(states) / 252
    
    return {
        'Switches/Year': f"{switches/years:.1f}",
        'Crisis %': f"{states.mean()*100:.1f}%",
        'Total Switches': switches
    }

In [10]:
train_period = turbulence.iloc[252:252*4]  # Year 2-4 (after 1yr lookback)
threshold_95 = train_period.quantile(0.95)

In [11]:
turb_states = (turbulence > threshold_95).astype(int)
turb_states = turb_states.dropna()

In [12]:
print("BASELINE: Kritzman Turbulence (95th Percentile Threshold)")
print("="*70)
print(f"Threshold: {threshold_95:.2f}")
print(f"\nCrisis Detection:")
turb_crisis = evaluate_crisis_detection(turb_states, crisis_events)
for event, rate in turb_crisis.items():
    print(f"  {event}: {rate}")

print(f"\nRegime Stability:")
turb_metrics = calculate_regime_metrics(turb_states)
for metric, value in turb_metrics.items():
    print(f"  {metric}: {value}")


In [13]:
def simple_volatility_baseline(returns, window=21, percentile=0.95):
    """
    Simplest possible baseline: rolling volatility threshold
    
    Parameters:
    -----------
    returns : pd.Series
        Daily returns
    window : int
        Rolling window for volatility (21 = 1 month)
    percentile : float
        Threshold percentile
    """
    # Calculate rolling volatility
    rolling_vol = returns.rolling(window=window).std() * np.sqrt(252)  # Annualized
    
    # Train threshold on first 3 years
    train_vol = rolling_vol.iloc[window:window+252*3]
    threshold = train_vol.quantile(percentile)
    
    # Classify regimes
    states = (rolling_vol > threshold).astype(int)
    
    return states.dropna(), rolling_vol.dropna(), threshold

# Apply simple volatility baseline
vol_states, rolling_vol, vol_threshold = simple_volatility_baseline(
    returns[asset], 
    window=7, 
    percentile=0.95
)

print("="*70)
print("BASELINE 2: Simple Rolling Volatility (95th Percentile)")
print("="*70)
print(f"Window: 21 days (1 month)")
print(f"Threshold: {vol_threshold:.4f} (annualized)")

print(f"\nCrisis Detection:")
vol_crisis = evaluate_crisis_detection(vol_states, crisis_events)
for event, rate in vol_crisis.items():
    print(f"  {event}: {rate}")

print(f"\nRegime Stability:")
vol_metrics = calculate_regime_metrics(vol_states)
for metric, value in vol_metrics.items():
    print(f"  {metric}: {value}")

In [14]:
hmm_probs = model.smoothed_marginal_probabilities[1]  # Regime 1 (crisis)
hmm_states = (hmm_probs > 0.5).astype(int)
hmm_states = pd.Series(hmm_states, index=returns.index)

print("\n" + "="*70)
print("YOUR HMM MODEL: Simple Returns + Switching Variance")
print("="*70)
print(f"AIC: {model.aic:.1f}")
print(f"BIC: {model.bic:.1f}")
print(f"Log-Likelihood: {model.llf:.1f}")

print(f"\nCrisis Detection:")
hmm_crisis = evaluate_crisis_detection(hmm_states, crisis_events)
for event, rate in hmm_crisis.items():
    print(f"  {event}: {rate}")

print(f"\nRegime Stability:")
hmm_metrics = calculate_regime_metrics(hmm_states)
for metric, value in hmm_metrics.items():
    print(f"  {metric}: {value}")

In [15]:
comparison =  pd.DataFrame({
    'Volatility MA': {**vol_crisis, **vol_metrics},
    'Turbulence 95th': {**turb_crisis, **turb_metrics},
    'HMM': {**hmm_crisis, **hmm_metrics, 
                   'AIC': f"{model.aic:.1f}", 
                   'BIC': f"{model.bic:.1f}"}
})
comparison

In [16]:
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# 1. SPY Returns
axes[0].plot(returns.index, returns.values, alpha=0.7, linewidth=0.5, color='black')
axes[0].axhline(0, color='red', linestyle='--', linewidth=0.8, alpha=1)
axes[0].set_ylabel('SPY Returns')
axes[0].set_title('Market Returns (SPX) - Blue Shading = HMM Crisis Detection', 
                  fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Add red shading for HMM crisis periods on first plot only
for i in range(len(hmm_states)):
    if hmm_states.iloc[i] == 1:
        # Find contiguous crisis periods
        if i == 0 or hmm_states.iloc[i-1] == 0:
            start_date = hmm_states.index[i]
        if i == len(hmm_states)-1 or hmm_states.iloc[i+1] == 0:
            end_date = hmm_states.index[i]
            axes[0].axvspan(start_date, end_date, alpha=0.2, color='blue', zorder=-1)

# 2. Simple Volatility
axes[1].plot(rolling_vol.index, rolling_vol.values, color='purple', linewidth=1)
axes[1].axhline(vol_threshold, color='purple', linestyle='--', 
                label=f'95th %ile = {vol_threshold:.4f}')
axes[1].set_ylabel('Volatility (Ann.)')
axes[1].set_title('Rolling Volatility (21-day)', fontsize=12, fontweight='bold')
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

# 3. Turbulence Score
axes[2].plot(turbulence.index, turbulence.values, color='orange', linewidth=1)
axes[2].axhline(threshold_95, color='orange', linestyle='--', 
                label=f'95th %ile = {threshold_95:.1f}')
axes[2].set_ylabel('Turbulence')
axes[2].set_title('Kritzman Turbulence Index', fontsize=12, fontweight='bold')
axes[2].legend(loc='upper left')
axes[2].grid(True, alpha=0.3)

# 4. Comparison: All three regimes overlaid
axes[3].fill_between(vol_states.index, 0, vol_states*0.33, 
                     alpha=0.5, color='purple', label='Volatility')
axes[3].fill_between(turb_states.index, 0.34, 0.34 + turb_states*0.33, 
                     alpha=0.5, color='orange', label='Turbulence')
axes[3].fill_between(hmm_states.index, 0.67, 0.67 + hmm_states*0.33, 
                     alpha=0.5, color='blue', label='HMM')
axes[3].set_ylabel('Model')
axes[3].set_yticks([0.165, 0.5, 0.835])
axes[3].set_yticklabels(['Volatility', 'Turbulence', 'HMM'])
axes[3].set_title('Regime Detection Comparison', fontsize=12, fontweight='bold')
axes[3].legend(loc='upper left', ncol=3)
axes[3].grid(True, alpha=0.3)
axes[3].set_ylim(0, 1)
axes[3].set_xlabel('Date')

plt.tight_layout()
plt.show()

In [ ]:
# Cell: Calculate switches with different consecutive-day filters

def apply_consecutive_filter(states, n_days=2):
    """
    Apply n-day consecutive filter to regime states
    
    Parameters:
    -----------
    states : pd.Series
        Binary regime states (0=normal, 1=crisis)
    n_days : int
        Number of consecutive days required before switching
    
    Returns:
    --------
    filtered_states : pd.Series
        Filtered regime states
    """
    filtered = states.copy()
    
    # Start with first state
    current_regime = states.iloc[0]
    filtered.iloc[0] = current_regime
    
    consecutive_count = 0
    
    for i in range(1, len(states)):
        if states.iloc[i] != current_regime:
            # Different regime detected
            consecutive_count += 1
            
            if consecutive_count >= n_days:
                # Switch confirmed after n consecutive days
                current_regime = states.iloc[i]
                consecutive_count = 0
            
            # Keep previous regime until confirmed
            filtered.iloc[i] = current_regime
        else:
            # Same regime, reset counter
            consecutive_count = 0
            filtered.iloc[i] = current_regime
    
    return filtered


# Apply filters with different thresholds
print("="*70)
print("REGIME SWITCHING ANALYSIS: CONSECUTIVE-DAY FILTERS")
print("="*70)

# Original HMM states (no filter)
original_switches = (hmm_states.diff() != 0).sum()
years = len(hmm_states) / 252

print(f"\nOriginal HMM (no filter):")
print(f"  Total switches: {original_switches}")
print(f"  Switches per year: {original_switches/years:.1f}")
print(f"  Crisis %: {hmm_states.mean()*100:.1f}%")

# Test different consecutive-day requirements
for n_days in [2, 3, 4]:
    filtered_states = apply_consecutive_filter(hmm_states, n_days=n_days)
    
    switches = (filtered_states.diff() != 0).sum()
    crisis_pct = filtered_states.mean() * 100
    
    print(f"\n{n_days}-Day Consecutive Filter:")
    print(f"  Total switches: {switches}")
    print(f"  Switches per year: {switches/years:.1f}")
    print(f"  Crisis %: {crisis_pct:.1f}%")
    
    # Check crisis detection still works
    covid_mask = (filtered_states.index >= '2020-02-20') & (filtered_states.index <= '2020-04-01')
    if covid_mask.sum() > 0:
        covid_detection = filtered_states[covid_mask].mean() * 100
        print(f"  COVID-19 detection: {covid_detection:.1f}%")


# Comparison table
print("\n" + "="*70)
print("COMPARISON TABLE")
print("="*70)

comparison_data = {
    'Filter': ['No filter', '2-day', '3-day', '4-day'],
    'Switches/Year': [],
    'Total Switches': [],
    'Crisis %': [],
    'COVID Detection': []
}

# Original
comparison_data['Switches/Year'].append(original_switches/years)
comparison_data['Total Switches'].append(original_switches)
comparison_data['Crisis %'].append(hmm_states.mean()*100)
covid_mask = (hmm_states.index >= '2020-02-20') & (hmm_states.index <= '2020-04-01')
comparison_data['COVID Detection'].append(hmm_states[covid_mask].mean()*100)

# Filtered versions
for n_days in [2, 3, 4]:
    filtered = apply_consecutive_filter(hmm_states, n_days=n_days)
    switches = (filtered.diff() != 0).sum()
    
    comparison_data['Switches/Year'].append(switches/years)
    comparison_data['Total Switches'].append(switches)
    comparison_data['Crisis %'].append(filtered.mean()*100)
    
    covid_mask = (filtered.index >= '2020-02-20') & (filtered.index <= '2020-04-01')
    comparison_data['COVID Detection'].append(filtered[covid_mask].mean()*100)

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

In [26]:
# Apply 2-day consecutive filter to HMM states
filtered_states_2day = apply_consecutive_filter(hmm_states, n_days=2)

# Align indices - use the returns/hmm_states index
aligned_close = close.reindex(filtered_states_2day.index)

# Split into regime-based dataframes with filtered states
df_1_filtered = aligned_close[filtered_states_2day == 0]  # Low variance (normal)
df_2_filtered = aligned_close[filtered_states_2day == 1]  # High variance (crisis)

# Ensure datetime index and sort
df_1_filtered.index = pd.to_datetime(df_1_filtered.index)
df_1_filtered = df_1_filtered.sort_index()
df_2_filtered.index = pd.to_datetime(df_2_filtered.index)
df_2_filtered = df_2_filtered.sort_index()

# Plot with 2-day filter applied
plt.figure(figsize=(15, 10))
plt.scatter(df_1_filtered.index, df_1_filtered, color='blue', 
            label="Low Variance Regime (2-day filter)", s=20, alpha=0.7)
plt.scatter(df_2_filtered.index, df_2_filtered, color='red', 
            label="High Variance Regime (2-day filter)", s=20, alpha=0.7)

# Add shaded region for backtesting period
plt.axvspan('2024-07-01', '2025-07-01', alpha=0.2, color='green', 
            zorder=-1, label="Backtesting Period")

plt.title("SPY Price - HMM Regime Classification (2-Day Consecutive Filter)", 
          fontsize=14, fontweight='bold')
plt.ylabel("Price ($)", fontsize=12)
plt.xlabel("Date", fontsize=12)
plt.legend(loc='upper left', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print summary stats
print("="*70)
print("2-DAY FILTER APPLIED")
print("="*70)
print(f"Total switches: {(filtered_states_2day.diff() != 0).sum()}")
print(f"Switches per year: {(filtered_states_2day.diff() != 0).sum() / (len(filtered_states_2day)/252):.1f}")
print(f"Crisis days: {filtered_states_2day.sum()} ({filtered_states_2day.mean()*100:.1f}%)")
print(f"Normal days: {(filtered_states_2day == 0).sum()} ({(filtered_states_2day == 0).mean()*100:.1f}%)")

In [21]:
filtered_states_2day

In [ ]:
close.loc[filtered_states_2day == 0]['SPX']

In [23]:
[filtered_states_2day == 0]